# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display
# Import necessary libraries
import folium

# Create a map
map_plot = folium.Map(location=[21.3469, -158.0858], zoom_start=4)  # Replace latitude and longitude with your preferred starting location

# Add a point for each city with the size representing humidity
for index, city in city_data_df.iterrows():
    
     # Get latitude, longitude, and humidity of the city
    lat = city['Lat']
    lon = city['Lng']
    humidity = city['Humidity']
    
    # Create a marker for the city with the size based on humidity
    marker = folium.CircleMarker(
        location=[lat, lon],
        radius=humidity/10,  # Adjust the divisor to change the size of the point based on humidity
        popup=f"{city['City']}, {city['Country']}<br>Humidity: {humidity}",
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    )
    
    # Add the marker to the map
    marker.add_to(map_plot)

# Display the map
map_plot  # This will display the map inline in Jupyter Notebook or your IDE

# If you are using Jupyter Notebook and want to save the map to an HTML file, you can use the following code:
# map_plot.save('humidity_map.html')


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Step 2: Narrow down cities that fit criteria and drop any results with null values

# Define the criteria for ideal weather conditions
ideal_conditions_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) &
    (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 4.5)
]

# Drop any rows with null values (if any)
ideal_conditions_df.dropna(inplace=True)

# Display sample data of cities that meet the ideal weather conditions
sample_data_df = ideal_conditions_df
sample_data_df.head()

C:\Users\tuanq\AppData\Local\Temp\ipykernel_21076\2184071442.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_conditions_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246


In [ ]:
========================================================================

### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Step 3: Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_conditions_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame to store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data of the hotel_df DataFrame
#print(hotel_df.head())
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
152,kahului,US,20.8947,-156.4700,60,
197,gat,IL,31.6100,34.7642,100,
211,laguna,US,38.4210,-121.4238,79,
240,tikaitnagar,IN,26.9500,81.5833,59,
265,san quintin,MX,30.4833,-115.9500,74,
340,santa rosalia,MX,27.3167,-112.2833,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

import requests

# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "type": "hotel",
    "limit": 1,
    "apiKey": "geoapify_key"  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    city_lat = row['Lat']
    city_lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distanceToPoint([{city_lng}, {city_lat}]) lt {radius}"
    params["bias"] = f"proximity([{city_lng}, {city_lat}])"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)


Starting hotel search
kapaa - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
kahului - nearest hotel: No hotel found
gat - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: No hotel found
santa rosalia - nearest hotel: No hotel found
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel found
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: No hotel found
capoterra - nearest hotel: No hotel found
stolac - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
               City Country      Lat       Lng  Humidity      Hotel Name
45            kapaa      US  22.0752 -159.3190        84  No hotel found
51             hilo      US  19.7297 -155.0900        83  No hotel found
63            banda      IN  25.4833   80.3333    

In [27]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,No hotel found
51,hilo,US,19.7297,-155.0900,83,No hotel found
63,banda,IN,25.4833,80.3333,52,No hotel found
81,makakilo city,US,21.3469,-158.0858,81,No hotel found
152,kahului,US,20.8947,-156.4700,60,No hotel found
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,No hotel found
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,No hotel found
340,santa rosalia,MX,27.3167,-112.2833,56,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

In [ ]:
# Import necessary libraries
import folium
from geopy.geocoders import Nominatim

# Step 1: Create a map
#map_plot = folium.Map(location=[21.3469, -158.0858], zoom_start=4)  

# Import necessary libraries
import folium
from geopy.geocoders import Nominatim

# Step 1: Create a map
map_plot = folium.Map(location=[21.3469, -158.0858], zoom_start=4)  


# Step 2: Add a point for each city with the size representing humidity
for index, city in city_data.iterrows():
    # Get latitude, longitude, and humidity of the city
    lat = city['Lat']
    lon = city['Lng']
    humidity = city['Humidity']
    
    # Perform reverse geocoding to get the hotel name and country information
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.reverse((lat, lon), exactly_one=True)
    hotel_name = location.raw.get('hotel', 'N/A')
    country = location.raw.get('address', {}).get('country', 'N/A')
    
    # Create a marker for the city with the size based on humidity and add hotel and country information to the popup
    marker = folium.CircleMarker(
        location=[lat, lon],
        radius=humidity/10,  # Adjust the divisor to change the size of the point based on humidity
        popup=f"{city['City']}, {country}<br>Hotel: {hotel_name}<br>Humidity: {humidity}",
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    )
    
    # Add the marker to the map
    marker.add_to(map_plot)

# Step 3: Display the map
map_plot  # This will display the map inline in Jupyter Notebook or your IDE

# If you are using Jupyter Notebook and want to save the map to an HTML file, you can use the following code:
# map_plot.save('humidity_map.html')